In [ ]:
import torch
from torch import nn as nn
from torch.utils import data
import numpy as np
import gym


In [ ]:
env = gym.make('CartPole-v0')

In [ ]:
env.observation_space.sample()

array([-2.9520917e+00,  2.4096669e+38,  1.7626123e-01, -1.8542970e+38],
      dtype=float32)

In [ ]:
env.action_space.sample()

1

In [ ]:
s = env.observation_space.sample()
a = np.array([env.action_space.sample()])
sa = np.concatenate([s,a])
sa

array([-4.36536551e+00,  3.17697194e+38, -2.05596402e-01,  1.10934700e+38,
        0.00000000e+00])

In [ ]:
torch.from_numpy(s)

tensor([-4.3654e+00,  3.1770e+38, -2.0560e-01,  1.1093e+38])

In [ ]:

a = qlearn(torch.from_numpy(s).float())
a

tensor([-0.0449, -0.0274], grad_fn=<AddBackward0>)

In [ ]:
torch.nn.functional.relu()

TypeError: ignored

In [ ]:
print(a)

None


In [ ]:
s.size

4

In [ ]:
class dqn(nn.Module):
  def __init__(self):
    super(dqn,self).__init__()
    self.fc1 = nn.Linear(4,256)
    self.fc2 = nn.Linear(256,2)

  def forward(self,s):
    output = torch.nn.functional.relu((self.fc1(s)))
    output = self.fc2(output)
    return output
qlearn = dqn()
  

In [ ]:
qlearn.train()
optimizer = torch.optim.SGD(qlearn.parameters(),lr=0.01, momentum=0.9)

In [ ]:
class TD_0_dqn():
  def __init__(self):
    self.q = dqn()
    self.history = []
    self.gamma = 0.99
    self.w_e = 0.0004
    self.max_e = 0.2
    self.loss = nn.MSELoss()
    self.optimizer = torch.optim.SGD(self.q.parameters(),lr=0.01, momentum=0.9)

  def act(self,s):
    a = torch.argmax(self.q(torch.from_numpy(s).float()))
    return a.item()
  
  def e_greedy(self,s,e):
    if np.random.rand() < e:
      a = env.action_space.sample()
    else:
      a = self.act(s)
    return a

  def decay(self,episode):
    e = max(-episode * self.w_e + self.max_e, 0.1/(episode + 1))
    return e

  def update(self,s,a,s1,r,a1):
    next_state_estimation = self.q(torch.from_numpy(s).float())[a]
    next_state_max = self.gamma * self.q(torch.from_numpy(s1).float())[a1] + r
    l = self.loss(next_state_estimation,next_state_max)
    #print(l)
    self.optimizer.zero_grad()
    l.backward()
    self.optimizer.step()
    return l

In [ ]:
agent = TD_0_dqn()
n_episodes = 500

In [ ]:
for episoda in range(n_episodes):
  obs = env.reset()
  e = agent.decay(episoda)
  print(e)
  #e = 0.1
  s = obs
  a = agent.e_greedy(s,e)
  G = 0
  L = 0
  #for t in range(500):
  while True:
    obs, r, done, _ = env.step(a)
    G += r
    s1 = obs
    a1 = agent.e_greedy(s1,e)
    l = agent.update(s,a,s1,r,a1)
    L += l
    if done:
      agent.history.append(G)
      #agent.policy_inprovement()
      print("Episoda: {} | Final return: {} | average loss: {}".format(episoda,G,L.item()/G))
      break
    s,a = s1,a1

0.2
Episoda: 0 | Final return: 16.0 | average loss: 1.8483415842056274
0.1996
Episoda: 1 | Final return: 16.0 | average loss: 1.6647886037826538
0.19920000000000002
Episoda: 2 | Final return: 11.0 | average loss: 1.817173177545721
0.1988
Episoda: 3 | Final return: 9.0 | average loss: 0.2717168066236708
0.19840000000000002
Episoda: 4 | Final return: 18.0 | average loss: 2.456307305230035
0.198
Episoda: 5 | Final return: 25.0 | average loss: 1.2675244903564453
0.1976
Episoda: 6 | Final return: 65.0 | average loss: 1.2893858689528246
0.19720000000000001
Episoda: 7 | Final return: 27.0 | average loss: 1.0598158659758392
0.1968
Episoda: 8 | Final return: 13.0 | average loss: 0.9868921866783729
0.19640000000000002
Episoda: 9 | Final return: 27.0 | average loss: 1.0912253768355757
0.196
Episoda: 10 | Final return: 12.0 | average loss: 1.0499247709910076
0.19560000000000002
Episoda: 11 | Final return: 51.0 | average loss: 1.0981710845348882
0.1952
Episoda: 12 | Final return: 37.0 | average los

In [ ]:
test = []
for i in range(10):
  obs = env.reset()
  s = obs
  a = agent.act(s)
  G = 0
  while True:
    obs, r, done, _ = env.step(a)
    G += r
    s1 = obs
    a = agent.act(s1)
    if done:
      test.append(G)
      break
print("off Qlearn | average score: {}".format(np.mean(test)))

off Qlearn | average score: 200.0
